In [13]:
#### select ASm6A sites for validation
#### There are two independent data sets

os.chdir("/Charles/project/ASm6A/data/validation_ASm6A/LBFC20201232/count/")
in_file = "ASm6As.bed"
prefix = "ASm6As"
##
# os.chdir("/Charles/project/ASm6A/ASm6A/corrRBP/motifbreakR/FUS_related_ASm6As/")
# in_file = "totalSamples_positive.bed"
# prefix = "totalSamples-ASm6As"
###
flank_bed = "%s_flank.bed" % prefix
# result_overlapFUS_bed = "%s_flank_FUS.bed" % prefix
# result_overlapFUS_overlapPeak_bed = "%s_flank_FUS_Peak.bed" % prefix
# final_result = "%s_finalResults.bed" % prefix

fus = "/Charles/project/ASm6A/data/RBP/union/union.bed"
#     fus = "/Charles/project/ASm6A/data/RBP/union/ContainingFUS_genes.bed"
m6a_peak = "/Charles/project/ASm6A/shFUS/mapping/human/peak/merged_peak_MSPC/wt1/ConsensusPeaks.bed"
#     m6a_peak = "/Charles/project/ASm6A/shFUS/mapping/human/peak/merged_peak_MSPC/normalized_macs/wt1.bed"

def extract_flankRegion(in_file):
    df = pd.read_table(in_file, header=None)
    df['chr'] = df.iloc[:, 0]
    df['start'] = df.iloc[:, 1] - 100
    df['end'] = df.iloc[:, 2] + 100
    df['name'] = df.iloc[:, 3]
    df[['chr', 'start', 'end', 'name']].drop_duplicates().to_csv(result_bed,
                                                         sep="\t",
                                                         header=False,
                                                         index=False)

def overlap_bed(bed1, bed2, result_bed):
    os.system("bedtools intersect -a %s -b %s -wa -wb > %s" %
              (bed1, bed2, result_bed))
# def overlap_FUS():
#     os.system("bedtools intersect -a %s -b %s -wa -wb > %s" %
#               (result_bed, fus, result_overlapFUS_bed))

# def overlap_293T():
#     os.system(
#         "bedtools intersect -a %s -b %s -wa -wb > %s" %
#         (result_overlapFUS_bed, m6a_peak, result_overlapFUS_overlapPeak_bed))

def calc_dist(in_file, out_file):
    with open(out_file, 'w') as fw:
        with open(in_file, 'r') as f:
            for line in f.readlines():
                ASm6A_position = int(line.split("\t")[2]) - 100 ####
                FUS_start = int(line.split("\t")[6])
                FUS_end = int(line.split("\t")[7])
                v_list = [int(x) for x in range(FUS_start+1, FUS_end+1)]
                distance = min([abs(x-ASm6A_position) for x in v_list])
                fw.write(line.strip() + "\t%s\n" % distance)
    
# def extract_fq():
#     genome = "/home/galaxy/project/alleleSpecific_analysis/data/hg19_genome/hg19.fa"
#     result_fa = result_overlapFUS_overlapPeak_bed.replace(".bed", ".fa")
#     os.system("bedtools getfasta -fi %s -bed %s -name+ > %s" %
#               (genome, result_overlapFUS_overlapPeak_bed, result_fa))
def extract_fq(bed, result_file):
    genome = "/home/galaxy/project/alleleSpecific_analysis/data/hg19_genome/hg19.fa"
    result_fa = os.path.basename(bed).replace(".bed", ".fa")
    os.system("bedtools getfasta -fi %s -bed %s -name+ > %s" %
              (genome, bed, result_fa))
    with open(result_file, 'w') as fw:
        with open(result_fa, 'r') as f:
            contents = f.readlines()
            for i in range(len(contents)-1):
                if contents[i].startswith(">"):
                    title = contents[i].strip()
                    seq = contents[i+1].strip()
                    chro = title.split("::")[1].split(":")[0]
                    start = title.split(":")[-1].split("-")[0]
                    end = title.split("-")[-1]
                    name = title.split(">",1)[1].split("::")[0]
                    new_line = ""
                    if name != "":
                        new_line = "%s\t%s\t%s\t%s\t%s" % (chro, start, end, name, seq)
                    else:
                        new_line = "%s\t%s\t%s\t%s" % (chro, start, end, seq)
                    fw.write(new_line + "\n")
    os.remove(result_fa)


extract_flankRegion(in_file)
extract_fq(flank_bed, "flank_withSeq.bed")
extract_fq(fus, "fus_withSeq.bed")
extract_fq(m6a_peak, "m6aPeak_withSeq.bed")

overlap_bed("flank_withSeq.bed", "fus_withSeq.bed", "fus_asm6a_withSeq.bed")
overlap_bed("fus_asm6a_withSeq.bed", "m6aPeak_withSeq.bed", "fus_asm6a_m6aPeak_withSeq.bed")
calc_dist("fus_asm6a_m6aPeak_withSeq.bed", "fus_asm6a_m6aPeak_results.bed")

#### pick out ASm6As from 90 tissue samples

In [26]:
os.chdir("/Charles/project/ASm6A/ASm6A/FUS_validate/")
####
fus = "/Charles/project/ASm6A/data/RBP/union/union.bed"
#     fus = "/Charles/project/ASm6A/data/RBP/union/ContainingFUS_genes.bed"
asm6a = "total.bed"
m6a_peak = "/Charles/project/ASm6A/shFUS/mapping/human/peak/merged_peak_MSPC/wt1/ConsensusPeaks.bed"

def overlap_bed(bed1, bed2, result_bed):
    os.system("bedtools intersect -a %s -b %s -wa -wb > %s" %
              (bed1, bed2, result_bed))
    
def extract_fq(bed, result_file):
    genome = "/home/galaxy/project/alleleSpecific_analysis/data/hg19_genome/hg19.fa"
    result_fa = os.path.basename(bed).replace(".bed", ".fa")
    os.system("bedtools getfasta -fi %s -bed %s -name+ > %s" %
              (genome, bed, result_fa))
    with open(result_file, 'w') as fw:
        with open(result_fa, 'r') as f:
            contents = f.readlines()
            for i in range(len(contents)-1):
                if contents[i].startswith(">"):
                    title = contents[i].strip()
                    seq = contents[i+1].strip()
                    chro = title.split("::")[1].split(":")[0]
                    start = title.split(":")[-1].split("-")[0]
                    end = title.split("-")[-1]
                    new_line = "%s\t%s\t%s\t%s" % (chro, start, end, seq)
                    fw.write(new_line + "\n")
#                     i += 1
    os.remove(result_fa)
                
extract_fq(fus, "fus_withSeq.bed")
# extract_fq(asm6a, "asm6a_withSeq.bed")
extract_fq(m6a_peak, "m6aPeak_withSeq.bed")

overlap_bed("fus_withSeq.bed", asm6a, "fus_asm6a_withSeq.bed")
overlap_bed("fus_asm6a_withSeq.bed", "m6aPeak_withSeq.bed", "fus_asm6a_m6aPeak_withSeq.bed")

In [31]:
### select motif
in_file = "fus_asm6a_m6aPeak_withSeq.bed"
with open("fus_asm6a_m6aPeak_withMotif.bed", 'w') as fw:
    with open(in_file, 'r') as f:
        for line in f.readlines():
            fus_seq = line.split("\t")[3]
            if ("GGTG" in fus_seq):
                asm6a_pos = int(line.split("\t")[6])
                fus_start = int(line.split("\t")[1])
                motif_index = fus_seq.index("GGTG")
                start = fus_start + motif_index + 1
                end = start + 3
                dist = min(abs(start-asm6a_pos), abs(end-asm6a_pos))
                fw.write(line.strip() + "\t%s\n" % str(dist))
            if ("CCAC" in fus_seq):
                asm6a_pos = int(line.split("\t")[6])
                fus_start = int(line.split("\t")[1])
                motif_index = fus_seq.index("CCAC")
                start = fus_start + motif_index + 1
                end = start + 3
                dist = min(abs(start-asm6a_pos), abs(end-asm6a_pos))
                fw.write(line.strip() + "\t%s\n" % str(dist))